# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Bring in data from csv
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data


,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,taoudenni,ML,0,1603217183,15,22.68,-3.98,88.02,9.15
1,1,senanga,ZM,0,1603217183,13,-16.12,23.27,83.25,5.14
2,3,nanortalik,GL,100,1603217184,85,60.14,-45.24,37.94,4.34
3,4,ringebu,NO,100,1603217184,94,61.52,10.17,30.99,3.00
4,5,tasiilaq,GL,75,1603217148,93,65.61,-37.64,32.00,20.80
...,...,...,...,...,...,...,...,...,...,...
611,658,camacha,PT,40,1603217380,64,33.08,-16.33,69.80,21.92
612,659,muravlenko,RU,100,1603217380,98,63.79,74.52,26.49,6.91
613,660,lakatoro,VU,82,1603217380,79,-16.10,167.42,76.89,7.31
614,661,biryusinsk,RU,100,1603217380,98,55.96,97.82,32.29,8.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

In [5]:
# Set intensity
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Windspeed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Country,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Windspeed
109,116,lukulu,ZM,0,1603217217,16,-14.37,23.24,78.96,5.57
111,118,poum,NC,0,1603217217,70,-20.23,164.02,74.12,5.21
120,128,bandarbeyla,SO,0,1603217218,74,9.49,50.81,79.84,3.87
135,146,riyadh,SA,0,1603217222,23,24.69,46.72,75.20,4.70
163,177,grand gaube,MU,0,1603217235,70,-20.01,57.66,75.99,8.99
195,213,amarpur,IN,0,1603217239,61,25.03,86.90,76.39,1.81
212,232,mumbwa,ZM,0,1603217251,18,-14.98,27.06,76.95,5.46
315,344,ambovombe,MG,0,1603217284,88,-25.17,46.08,70.77,3.18
375,408,pemba,MZ,0,1603217301,75,-12.97,40.52,78.39,7.23
557,601,sur,OM,0,1603217364,58,22.57,59.53,74.59,2.89


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
109,lukulu,ZM,-14.37,23.24,
111,poum,NC,-20.23,164.02,
120,bandarbeyla,SO,9.49,50.81,
135,riyadh,SA,24.69,46.72,
163,grand gaube,MU,-20.01,57.66,
195,amarpur,IN,25.03,86.90,
212,mumbwa,ZM,-14.98,27.06,
315,ambovombe,MG,-25.17,46.08,
375,pemba,MZ,-12.97,40.52,
557,sur,OM,22.57,59.53,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 109: lukulu.
Closest hotel in lukulu is Sandra Guest House.
------------
Retrieving Results for Index 111: poum.
Missing field/result... skipping.
------------
Retrieving Results for Index 120: bandarbeyla.
Closest hotel in bandarbeyla is Super secret pirate cove.
------------
Retrieving Results for Index 135: riyadh.
Closest hotel in riyadh is Best Western Plus Fursan Hotel.
------------
Retrieving Results for Index 163: grand gaube.
Closest hotel in grand gaube is LUX Grand Gaube Resort & Villas.
------------
Retrieving Results for Index 195: amarpur.
Missing field/result... skipping.
------------
Retrieving Results for Index 212: mumbwa.
Closest hotel in mumbwa is Madalitso Guest House.
------------
Retrieving Results for Index 315: ambovombe.
Closest hotel in ambovombe is Hotel Le Source.
------------
Retrieving Results for Index 375: pemba.
Closest hotel in pemba is Avani Pemba Beach Hotel.
------------
Retrieving Results for Index 557: sur.
Closest ho

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
109,lukulu,ZM,-14.37,23.24,Sandra Guest House
111,poum,NC,-20.23,164.02,
120,bandarbeyla,SO,9.49,50.81,Super secret pirate cove
135,riyadh,SA,24.69,46.72,Best Western Plus Fursan Hotel
163,grand gaube,MU,-20.01,57.66,LUX Grand Gaube Resort & Villas
195,amarpur,IN,25.03,86.90,
212,mumbwa,ZM,-14.98,27.06,Madalitso Guest House
315,ambovombe,MG,-25.17,46.08,Hotel Le Source
375,pemba,MZ,-12.97,40.52,Avani Pemba Beach Hotel
557,sur,OM,22.57,59.53,Sur Plaza Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Figure
fig


Figure(layout=FigureLayout(height='420px'))